In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import mediapipe as mp
import csv
import os
import emojis
import emoji

In [2]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic

# Make Detections

In [3]:
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame=cap.read()
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable= False
        results=holistic.process(image)
        image.flags.writeable= True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        #face landmarks
        mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(230,47,99),thickness=1,circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(120,200,78),thickness=1,circle_radius=1))
        
        #Right hand landmarks
        mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(67,98,22),thickness=1,circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(180,22,99),thickness=1,circle_radius=1))
                                  
        #Left hand Landmarks
        mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(60,110,100),thickness=1,circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(150,130,180),thickness=1,circle_radius=1))
                                  
        #Pose Landmarks
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(225,255,211),thickness=1,circle_radius=1),
                                    mp_drawing.DrawingSpec(color=(160,111,101),thickness=1,circle_radius=1))
        
        

        cv2.imshow("WebCam",image)
                                  
        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()
        

# Capture Landmarks And Export To CSV

In [4]:
num_coord=len(results.pose_landmarks.landmark) + len(results.face_landmarks.landmark)
num_coord

501

In [5]:
landmarks=["class"]
for val in range(1,num_coord+1):
    landmarks+=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [6]:
landmarks[-1]

'v501'

In [7]:
with open('coords.csv',mode='w',newline='') as f:
    csv_writer=csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [16]:
class_name="Victorius"

In [17]:
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame=cap.read()
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable= False
        results=holistic.process(image)
        image.flags.writeable= True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        #face landmarks
        mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(230,47,99),thickness=1,circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(120,200,78),thickness=1,circle_radius=1))
        
        #Right hand landmarks
        mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(67,98,22),thickness=1,circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(180,22,99),thickness=1,circle_radius=1))
                                  
        #Left hand Landmarks
        mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(60,110,100),thickness=1,circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(150,130,180),thickness=1,circle_radius=1))
                                  
        #Pose Landmarks
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(225,255,211),thickness=1,circle_radius=1),
                                    mp_drawing.DrawingSpec(color=(160,111,101),thickness=1,circle_radius=1))
        
        
        #Export coordinates
        try:
            #Extracting pose landmarks
            pose=results.pose_landmarks.landmark
            pose_row= list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())

            #Extracting face landmarks
            face=results.face_landmarks.landmark
            face_row=list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in face]).flatten())

            #Concat rows
            row=pose_row+face_row

            #Append classname
            row.insert(0,class_name)

            #Export to csv
            with open('coords.csv',mode='a',newline='') as f:
                csv_writer=csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
        except:
            pass

        cv2.imshow("WebCam",image)
                                  
        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

# Train Models

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
df=pd.read_csv('coords.csv')
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
785,Victorius,0.513724,0.619876,-1.064237,0.999868,0.530004,0.553548,-1.054023,0.999690,0.546004,...,-0.007502,0.0,0.558524,0.522090,0.005491,0.0,0.561393,0.517242,0.005284,0.0
786,Victorius,0.515276,0.619911,-1.058931,0.999872,0.531911,0.554128,-1.048224,0.999698,0.547725,...,-0.006556,0.0,0.558026,0.523569,0.007010,0.0,0.560912,0.518612,0.006953,0.0
787,Victorius,0.515538,0.619904,-1.068489,0.999879,0.532423,0.554255,-1.055229,0.999711,0.548289,...,-0.005720,0.0,0.559399,0.524142,0.007282,0.0,0.562163,0.518925,0.007208,0.0
788,Victorius,0.515713,0.620050,-1.063778,0.999884,0.532537,0.554882,-1.050740,0.999724,0.548414,...,-0.006002,0.0,0.559325,0.523471,0.007376,0.0,0.562139,0.518584,0.007312,0.0
789,Victorius,0.515867,0.620345,-1.068487,0.999889,0.532909,0.555323,-1.055598,0.999735,0.548885,...,-0.005004,0.0,0.558117,0.525054,0.008541,0.0,0.561047,0.520113,0.008548,0.0


In [20]:
df[df['class']=='Sad']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
231,Sad,0.531118,0.706455,-1.014002,0.999956,0.551199,0.633661,-1.017841,0.999840,0.569071,...,-0.025300,0.0,0.576647,0.613799,-0.013493,0.0,0.580076,0.607060,-0.014065,0.0
232,Sad,0.530081,0.709257,-1.092522,0.999928,0.550439,0.634203,-1.087503,0.999764,0.567773,...,-0.026284,0.0,0.575440,0.613004,-0.016613,0.0,0.578970,0.607017,-0.017328,0.0
233,Sad,0.529755,0.712987,-1.076266,0.999900,0.550068,0.635355,-1.072621,0.999684,0.567194,...,-0.026912,0.0,0.574016,0.612571,-0.018770,0.0,0.577219,0.606568,-0.019734,0.0
234,Sad,0.529978,0.716898,-1.200401,0.999876,0.550216,0.637620,-1.191672,0.999598,0.567241,...,-0.022815,0.0,0.570185,0.624109,-0.014960,0.0,0.573402,0.617117,-0.015531,0.0
235,Sad,0.529965,0.717609,-1.153043,0.999858,0.550152,0.638460,-1.148694,0.999562,0.567127,...,-0.026915,0.0,0.573641,0.623403,-0.019213,0.0,0.576975,0.615899,-0.019989,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,Sad,0.561095,0.692528,-1.296951,0.999683,0.582183,0.621260,-1.275635,0.999104,0.598589,...,-0.020143,0.0,0.611757,0.619924,-0.007099,0.0,0.615568,0.613838,-0.007267,0.0
415,Sad,0.562118,0.698074,-1.336761,0.999669,0.583246,0.626900,-1.315045,0.999076,0.599654,...,-0.020311,0.0,0.611284,0.619095,-0.007875,0.0,0.615042,0.613005,-0.008045,0.0
416,Sad,0.561301,0.702489,-1.276098,0.999666,0.582655,0.630174,-1.248420,0.999065,0.599454,...,-0.018931,0.0,0.608531,0.624249,-0.005845,0.0,0.612294,0.618545,-0.006012,0.0
417,Sad,0.560318,0.701740,-1.286895,0.999660,0.581684,0.629908,-1.264745,0.999033,0.598539,...,-0.020080,0.0,0.608405,0.622894,-0.007317,0.0,0.612299,0.616980,-0.007503,0.0


In [21]:
X=df.drop('class',axis=1)
y=df['class']

In [22]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1234)

In [23]:
y_train

524          Wow
698    Victorius
366          Sad
636    Victorius
696    Victorius
         ...    
372          Sad
204        Happy
53         Happy
294          Sad
723    Victorius
Name: class, Length: 553, dtype: object

In [24]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier ,GradientBoostingClassifier

In [25]:
pipelines={
    'lr':make_pipeline(StandardScaler(),LogisticRegression()),
    'rc':make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [26]:
fit_models={}
for algo,pipeline in pipelines.items():
    model=pipeline.fit(X_train,y_train)
    fit_models[algo]=model

C:\Users\VC\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [27]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [28]:
from sklearn.metrics import accuracy_score
import pickle

In [29]:
for algo,model in fit_models.items():
    yhat=model.predict(X_test)
    print(algo,accuracy_score(y_test,yhat))

lr 1.0
rc 1.0
rf 0.9915611814345991
gb 0.9873417721518988


In [30]:
fit_models['rf'].predict(X_test)

array(['Happy', 'Happy', 'Sad', 'Happy', 'Happy', 'Wow', 'Happy', 'Happy',
       'Victorius', 'Victorius', 'Sad', 'Happy', 'Wow', 'Sad',
       'Victorius', 'Victorius', 'Wow', 'Wow', 'Wow', 'Wow', 'Wow', 'Sad',
       'Happy', 'Wow', 'Happy', 'Wow', 'Happy', 'Sad', 'Sad', 'Victorius',
       'Sad', 'Victorius', 'Sad', 'Victorius', 'Wow', 'Sad', 'Happy',
       'Wow', 'Happy', 'Wow', 'Sad', 'Happy', 'Wow', 'Wow', 'Happy',
       'Sad', 'Victorius', 'Wow', 'Wow', 'Wow', 'Wow', 'Happy', 'Sad',
       'Happy', 'Sad', 'Wow', 'Sad', 'Victorius', 'Happy', 'Happy',
       'Happy', 'Victorius', 'Happy', 'Happy', 'Sad', 'Happy', 'Happy',
       'Victorius', 'Wow', 'Happy', 'Sad', 'Sad', 'Happy', 'Victorius',
       'Happy', 'Victorius', 'Victorius', 'Sad', 'Sad', 'Wow',
       'Victorius', 'Wow', 'Victorius', 'Wow', 'Victorius', 'Sad', 'Wow',
       'Happy', 'Wow', 'Happy', 'Happy', 'Happy', 'Sad', 'Happy', 'Wow',
       'Happy', 'Wow', 'Happy', 'Happy', 'Sad', 'Happy', 'Victorius',
       'Wo

In [31]:
y_test

146        Happy
227        Happy
262          Sad
169        Happy
40         Happy
         ...    
272          Sad
395          Sad
757    Victorius
375          Sad
728    Victorius
Name: class, Length: 237, dtype: object

In [32]:
with open('body_language.pkl','wb') as f:
    pickle.dump(fit_models['rf'],f)

# Make Detections

In [33]:
with open('body_language.pkl','rb') as f:
    model=pickle.load(f)

In [34]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [35]:
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame=cap.read()
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable= False
        results=holistic.process(image)
        image.flags.writeable= True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        #face landmarks
        mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(230,47,99),thickness=1,circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(120,200,78),thickness=1,circle_radius=1))
        
        #Right hand landmarks
        mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(67,98,22),thickness=1,circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(180,22,99),thickness=1,circle_radius=1))
                                  
        #Left hand Landmarks
        mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(60,110,100),thickness=1,circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(150,130,180),thickness=1,circle_radius=1))
                                  
        #Pose Landmarks
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(225,255,211),thickness=1,circle_radius=1),
                                    mp_drawing.DrawingSpec(color=(160,111,101),thickness=1,circle_radius=1))
        
        
        #Export coordinates
        try:
            #Extracting pose landmarks
            pose=results.pose_landmarks.landmark
            pose_row= list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())

            #Extracting face landmarks
            face=results.face_landmarks.landmark
            face_row=list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in face]).flatten())

            #Concat rows
            row=pose_row+face_row

#             #Append classname
#             row.insert(0,class_name)

#             #Export to csv
#             with open('coords.csv',mode='a',newline='') as f:
#                 csv_writer=csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row)


            X=pd.DataFrame([row])
            body_language_class=model.predict(X)[0]
            body_language_prob=model.predict_proba(X)[0]
            print(body_language_class,body_language_prob)
            
            coords=tuple(np.multiply(
                    np.array(
                    (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                     results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                ,[640,480]).astype(int))
            
            
            cv2.rectangle(image,
                         (coords[0],coords[1]+5),
                         (coords[0]+len(body_language_class)*20,coords[1]-30),
                          (230,21,100),-1)
            
            cv2.putText(image,body_language_class,coords,
                       cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1,cv2.LINE_AA)
            
            #Status box
            cv2.rectangle(image,(0,0),(250,60),(250,0,10),-1)
            
            cv2.putText(image,'PROB',(30,20),
                       cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(round(body_language_prob[np.argmax(body_language_prob)],2))
                       ,(30,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,'CLASS',(130,20),
                       cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,body_language_class.split(' ')[0]
                       , (130,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)
                         
        except:
            pass

        cv2.imshow("WebCam",image)
                                  
        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()
        

Sad [0.   0.71 0.06 0.23]
Sad [0.   0.46 0.08 0.46]
Sad [0.   0.46 0.08 0.46]
Wow [0.   0.43 0.08 0.49]
Wow [0.   0.39 0.07 0.54]
Wow [0.   0.41 0.06 0.53]
Wow [0.   0.42 0.05 0.53]
Sad [0.   0.52 0.01 0.47]
Sad [0.   0.52 0.   0.48]
Wow [0.   0.46 0.   0.54]
Wow [0.   0.47 0.01 0.52]
Sad [0.   0.53 0.   0.47]
Sad [0.   0.95 0.   0.05]
Sad [0.   0.95 0.   0.05]
Sad [0.   0.99 0.   0.01]
Sad [0.   0.98 0.01 0.01]
Sad [0.   0.98 0.01 0.01]
Sad [0.   0.98 0.01 0.01]
Sad [0.   0.98 0.01 0.01]
Sad [0.   0.98 0.01 0.01]
Sad [0.   0.98 0.01 0.01]
Sad [0.   0.98 0.01 0.01]
Sad [0.   0.98 0.01 0.01]
Sad [0.   0.98 0.01 0.01]
Sad [0.   0.98 0.01 0.01]
Sad [0.   0.97 0.02 0.01]
Sad [0.   0.98 0.01 0.01]
Sad [0.   0.98 0.01 0.01]
Sad [0.   0.61 0.01 0.38]
Sad [0.   0.62 0.01 0.37]
Sad [0.   0.7  0.01 0.29]
Sad [0.   0.69 0.01 0.3 ]
Sad [0.   0.62 0.01 0.37]
Wow [0.03 0.33 0.15 0.49]
Wow [0.   0.29 0.19 0.52]
Wow [0.   0.29 0.17 0.54]
Wow [0.   0.29 0.13 0.58]
Wow [0.   0.27 0.16 0.57]
Wow [0.   0.